In [1]:

import re
import os
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from time import sleep
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from konlpy.tag import Komoran

from transformers import AutoTokenizer
from datetime import datetime
import dateparser

c:\Users\USER\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
import os

directories = [
    'C:/saltlux/7_17/200tokens_under',
    'C:/saltlux/7_17/256tokens',
    'C:/saltlux/7_17/500tokens',
    'C:/saltlux/7_17/1000tokens',
    'C:/saltlux/7_17/1500tokens',
    'C:/saltlux/7_17/2048tokens',
    'C:/saltlux/7_17/3000tokens',
    'C:/saltlux/7_17/3300tokens_over'
    
]

for directory in directories:
    if not os.path.exists(directory):
        os.makedirs(directory)

In [5]:
tokenizer = AutoTokenizer.from_pretrained("C:/saltlux/luxiagpt_tokenizer")

driver_path = "C:/pythonDir/chromedriver_win32/chromedriver.exe"
driver = webdriver.Chrome()
# tag = Komoran()
# windows = driver.window_handles
# driver = webdriver.Chrome('/chromedriver.exe')


cutoff_date = datetime.strptime('2023-07-14', '%Y-%m-%d')

xpath_string = '//*[@id="wrapArticleInfo"]/span[4]'

url= 'http://brunch.co.kr'
# URL에 접근
driver.get(url)
time.sleep(3)


time.sleep(7)

# last_crawled_page.txt 파일에서 가장 최근에 크롤링한 페이지 번호를 가져옵니다.
try:
    with open('last_crawled_page.txt', 'r') as f:
        lines = f.readlines()
        start_i = int(lines[0])
        start_j = int(lines[1])
except FileNotFoundError:
    start_i = 1
    start_j = 1

for i in range(11,24):
    driver.find_element(By.XPATH, f'//*[@id="mArticle"]/div[3]/div[1]/div/a[{i}]').click()
    # driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[3]/div[1]/div/a[2]').click()
    time.sleep(7)
    driver.switch_to.window(driver.window_handles[-1])


    for j in range(1,101):
        driver.find_element(By.XPATH, f'//*[@id="wrapArticle"]/div[1]/ul/li[{j}]/a/div[1]').click()
        driver.switch_to.window(driver.window_handles[-1])
        time.sleep(7)
        # 각 요소의 텍스트를 출력
        elements = driver.find_elements(By.CLASS_NAME, 'wrap_item.item_type_text')
        all_texts = []

        date_str = driver.find_element(By.XPATH, '//*[@id="wrapArticleInfo"]/span[4]').text

        try:
            # date_str이 '2023.07.14' 형식인 경우를 처리
            page_date = datetime.strptime(date_str, '%Y.%m.%d')
        except ValueError:
            # date_str이 '4분전', '1시간 전' 등의 형식인 경우를 처리
            page_date = datetime.now()

        # 페이지 날짜 정보 추출
        # date_str = driver.find_element(By.XPATH, '//*[@id="wrapArticleInfo"]/span[4]').text
        # page_date = datetime.strptime(date_str, '2023.07.14')  # 날짜 형식에 맞게 수정하세요

        # 페이지의 날짜가 기준 날짜보다 이전인 경우 루프를 종료
        print(page_date)
        if page_date < cutoff_date:
            break

        for element in elements:
            text = element.text
            all_texts.append(text)
        
        # 모든 텍스트를 하나의 문자열로 합칩니다.
        all_text = ' '.join(all_texts)
        try:
            # 토큰 수 확인
            num_tokens = len(tokenizer.tokenize(all_text))
        except Exception as e:
            print(f"Error occurred at Page {i}_{j} while tokenizing: {e}")
            num_tokens = 0

        print(f'Page {i}_{j} Token 수: {num_tokens}')

        # 텍스트 파일에 저장
        if num_tokens <= 200:
            path = 'C:/saltlux/7_17/200tokens_under'
        elif num_tokens <= 400:
            path = 'C:/saltlux/7_17/256tokens'
        elif num_tokens <= 750:
            path = 'C:/saltlux/7_17/500tokens'
        elif num_tokens <= 1250:
            path = 'C:/saltlux/7_17/1000tokens'
        elif num_tokens <= 1750:
            path = 'C:/saltlux/7_17/1500tokens'
        elif num_tokens <= 2300:
            path = 'C:/saltlux/7_17/2048tokens'
        elif num_tokens <= 3300:
            path = 'C:/saltlux/7_17/3000tokens'
        else:
            path = 'C:/saltlux/7_17/3300tokens_over'

        # 텍스트 파일에 저장
        with open(f'{path}/Page_{i}_{j}_{num_tokens}tokens.txt', 'w', encoding='utf-8') as f:
            f.write(all_text)

        # 크롤링한 마지막 페이지 번호를 기록합니다.
        with open('last_crawled_page.txt', 'w') as f:
            f.write(str(i) + '\n')
            f.write(str(j) + '\n')

        driver.close()
        driver.switch_to.window(driver.window_handles[-1])
    driver.close()
    driver.switch_to.window(driver.window_handles[-1])

    start_j = 1  # i가 증가할 때 j를 1로 재설정



2023-07-17 14:43:47.587243
Page 11_1 Token 수: 1514
2023-07-17 14:43:54.999690
Page 11_2 Token 수: 503
2023-07-17 14:44:02.276391
Page 11_3 Token 수: 418
2023-07-17 14:44:09.505353
Page 11_4 Token 수: 393
2023-07-17 14:44:17.028047
Page 11_5 Token 수: 2199
2023-07-17 14:44:24.431177
Page 11_6 Token 수: 1150
2023-07-17 14:44:31.799130
Page 11_7 Token 수: 1618
2023-07-17 14:44:39.216670
Page 11_8 Token 수: 1575
2023-07-17 14:44:46.720929
Page 11_9 Token 수: 808
2023-07-17 14:44:54.197636
Page 11_10 Token 수: 1153
2023-07-17 14:45:01.657711
Page 11_11 Token 수: 926
2023-07-17 14:45:08.983306
Page 11_12 Token 수: 1268
2023-07-17 14:45:16.433219
Page 11_13 Token 수: 608
2023-07-17 14:45:30.904527
Page 11_14 Token 수: 2564
2023-07-17 14:45:38.355514
Page 11_15 Token 수: 1271
2023-07-17 14:45:45.700165
Page 11_16 Token 수: 2819
2023-07-17 14:46:31.304064
Page 11_17 Token 수: 759
2023-07-17 14:46:38.846499
Page 11_18 Token 수: 3144
2023-07-17 14:46:46.207053
Page 11_19 Token 수: 1282
2023-07-17 14:46:53.568704
P

* 크롤링 하다 끊기더라도 마지막 페이지 기억하려했는데 무한스크롤이라 의미가 없음, 스크롤을 미리 해놓으면 클릭이 안됨. 누가 해결좀.

In [ ]:
# 날짜

//*[@id="wrapArticleInfo"]/span[4]


In [1]:
import os
import pandas as pd


data = []


directory = "C:/saltlux/7_17/1000tokens"


for filename in os.listdir(directory):
    num_tokens = int(filename.split('_')[-1].replace('tokens.txt', ''))
    if 1000 <= num_tokens < 1300:
        with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
            text = file.read()
        data.append([text, num_tokens])

df = pd.DataFrame(data, columns=['Text', 'NumTokens'])

In [12]:
df.to_excel('브런치크롤링.xlsx', index=False)

# 뉴스크롤링

In [4]:
from newspaper import Article

from tqdm import tqdm

from transformers import AutoTokenizer




tokenizer = AutoTokenizer.from_pretrained("C:/saltlux/luxiagpt_tokenizer")

# text_ls=[]

# token_ls=[]

# 파싱할 뉴스 기사 주소

for i in range(0,10):

    for j in tqdm(range(10)):

        for k in range(10):

            # https://www.lawtimes.co.kr/opinion/189182 함

            # https://www.lawtimes.co.kr/news/189207 0,1,2 함

            # https://n.news.naver.com/article/243/0000047500 0-4 까지

            # https://n.news.naver.com/article/001/0014067349 0000-7000 # 만단위도 가능 1~6

            # https://n.news.naver.com/mnews/hotissue/article/015/0004867921 6000-7000

            url = f'https://n.news.naver.com/article/001/0014062{i}{j}{k}' #1 , 0

            try:

                # 언어가 한국어이므로 language='ko'로 설정

                article = Article(url, language='ko')

                article.download()

                article.parse()




                text = article.text

                token_len = len(tokenizer.tokenize(text))

               

                if token_len >=1000 and token_len < 1200:

                    print('추가')

                    text_ls.append(text)

                    token_ls.append(token_len)

            except :

                # print('없음')

                pass

ModuleNotFoundError: No module named 'newspaper'

In [ ]:
import pandas as pd

df=pd.DataFrame({'doc':text_ls, 'token':token_ls})

 

df.to_excel('뉴스모음6.xlsx')